# 01. 파이토치를 이용한 텍스트 분류(Text classification using PyTorch)


이번 챕터에서는 파이토치(PyTorch)로 인공 신경망을 이용한 텍스트 분류를 실습합니다. 실습에 앞서 딥 러닝을 이용해서 텍스트 분류가 수행될 때, 어떤 작업과 구성으로 진행되는지 간단히 미리 정리해보겠습니다.

## 1. 훈련 데이터에 대한 이해
앞으로 배우게 될 텍스트 분류 작업은 __지도 학습(Supervised Learning)__에 속합니다. 지도 학습의 훈련 데이터는 레이블이라는 이름의 미리 정답이 적혀있는 데이터로 구성되어 있습니다. 쉽게 비유하면, 기계는 정답이 적혀져 있는 문제지를 열심히 공부하고, 향후에 정답이 없는 문제에 대해서도 정답을 예측해서 대답하게 되는 메커니즘입니다.

예를 들어 스팸 메일 분류기의 훈련 데이터같은 경우에는 메일의 내용과 해당 메일이 정상 메일인지, 스팸 메일인지 적혀있는 레이블로 구성되어져 있습니다. 아래와 같은 형식의 메일 샘플이 약 20,000개 있다고 가정해봅시다.


|텍스트(메일의 내용)|	레이블(스팸 여부)|
|:--:|:--:|
|당신에게 드리는 마지막 혜택! ...	|스팸 메일|
|내일 뵐 수 있을지 확인 부탁...	|정상 메일|
|쉿! 혼자 보세요...	|스팸 메일|
|언제까지 답장 가능할...	|정상 메일|
|(광고) 멋있어질 수 있는...	|스팸 메일|


20,000개의 메일 샘플을 가진 이 데이터는 메일의 내용을 담고 있는 텍스트 데이터와 이 데이터가 스팸 메일인지 아닌지가 적혀있는 레이블. 두 가지 열로 이루어져있습니다. 기계는 이 20,000개의 메일 샘플 데이터를 학습하게 되는데, 만약 데이터가 깔끔하고 모델 또한 잘 설계되어져 있다면 학습이 다 된 이 모델은 훈련 데이터에서는 없었던 어떤 메일 텍스트가 주어졌을 때 레이블을 예측하게 됩니다.

![](https://wikidocs.net/images/page/24873/%EB%B6%84%EB%A5%98.png)


***
## 2. 훈련 데이터와 테스트 데이터
위에서는 20,000개의 메일 샘플을 전부 훈련에 사용한다고 했지만 사실 갖고있는 전체 데이터를 전부 훈련에 사용하는 것 보다는 __테스트용__ 으로 일부는 남겨놓는 것으로 바람직합니다. 예를 들어서 20,000개의 샘플 중에서 18,000개의 샘플은 훈련용으로 사용하고, 2,000개의 샘플은 테스트용으로 보류한 채 훈련을 시킬 때는 사용하지 않을 수 있습니다. 그리고 나서 18,000개의 샘플로 모델이 훈련이 다 되면, 이제 보류해두었던 2,000개의 테스트용 샘플에서 레이블은 보여주지않고 모델에게 맞춰보라고 요구한 뒤, 정확도를 확인해볼 수 있습니다. 2,000개의 샘플에도 레이블이 있으므로 모델이 실제로 정답을 얼마나 맞추는지 정답률을 계산하게 됩니다.

뒤에 나오게 될 예제에서는 갖고 있는 데이터에서 분류하고자 하는 텍스트 데이터의 열을 X, 레이블 데이터의 열을 y라고 명명합니다. 그리고 이를 훈련 데이터(X_train, y_train)와 테스트 데이터(X_test, y_test)로 분리합니다. 모델은 X_train과 y_train을 학습하고, X_test에 대해서 레이블을 예측하게 됩니다. 그리고 모델이 예측한 레이블과 y_test를 비교해서 정답률을 계산하게 됩니다.

***
## 3. 단어에 대한 인덱스 부여
앞서 워드 임베딩 챕터에서 단어를 `밀집 벡터(dense vector)`로 바꾸는 워드 임베딩에 대해서 배운 바 있습니다. 8챕터와 9챕터에서 설명하였지만, 파이토치(PyTorch)의 nn.Embedding()은 단어 각각에 대해 정수가 맵핑된 입력에 대해서 임베딩 작업을 수행할 수 있게 해줍니다.

단어 각각에 숫자 맵핑, 인덱스를 부여하는 방법으로는 2챕터의 정수 인코딩 챕터에서와 같이 단어를 빈도수 순대로 정렬하고 순차적으로 인덱스를 부여하는 방법이 있습니다. 로이터 뉴스 분류하기와 IMDB 리뷰 감성 분류하기 챕터에서도 이 방법을 사용하였으며, 해당 챕터에서 사용할 데이터들은 이미 이 작업이 끝난 상태입니다.

등장 빈도순대로 단어를 정렬하여 인덱스를 부여하였을 때의 장점은 등장 빈도수가 적은 단어의 제거입니다. 예를 들어서 25,000개의 단어가 있다고 가정하고, 해당 단어를 등장 빈도수 순가 높은 순서로 1부터 25,000까지 인덱스를 부여했다고 해보겠습니다. 이렇게 되면 등장 빈도수 순대로 등수가 부여된 것과 다름없기 때문에 전처리 작업에서 1,000을 넘는 인덱스를 가진 단어들을 제거시켜버리면 등장 빈도수 상위 1,000개의 단어만 남길 수 있습니다.

***
## 4. RNN으로 분류하기

```
# 실제 RNN 은닉층을 추가하는 코드.
nn.RNN(input_size, hidden_size, batch_first=True)
```

텍스트 분류 관점에서 앞서 배운 RNN 코드의 timesteps와 input_dim, 그리고 hidden_size를 해석해보면 다음과 같습니다. (위의 코드에서는 바닐라 RNN을 사용했지만, RNN의 변형인 LSTM이나 GRU도 아래의 사항은 동일합니다.)

- hidden_size = 출력의 크기(output_dim).
- timesteps = 시점의 수 = 각 문서에서의 단어 수.
- input_size = 입력의 크기 = 각 단어의 벡터 표현의 차원 수.


## 5. RNN의 다-대-일(Many-to-One) 문제
텍스트 분류는 RNN의 다-대-일(Many-to-One) 문제에 속합니다. 즉, 텍스트 분류는 모든 시점(time step)에 대해서 입력을 받지만 최종 시점의 RNN 셀만이 은닉 상태를 출력하고, 이것이 출력층으로 가서 활성화 함수를 통해 정답을 고르는 문제가 됩니다.

이 때 두 개의 선택지 중에서 정답를 고르는 이진 분류(Binary Classification) 문제라고 하며, 세 개 이상의 선택지 중에서 정답을 고르는 다중 클래스 분류(Multi-Class Classification) 문제라고 합니다. 이 두 문제에서는 각각 문제에 맞는 다른 활성화 함수와 손실 함수를 사용할 것입니다.

이진 분류의 문제의 경우 출력층의 활성화 함수로 시그모이드 함수를, 다중 클래스 문제라면 출력층의 활성화 함수로 소프트맥스 함수를 사용합니다. 또한, 다중 클래스 분류 문제의 경우에는 클래스가 N개라면 출력층에 해당되는 밀집층(dense layer)의 크기는 N으로 합니다. 즉, 출력층의 뉴런의 수는 N개입니다. (하지만 소프트맥스 함수로 이진 분류를 할 수도 있습니다. 출력층에 뉴런을 2개로 배치하면 됩니다.)

이번 챕터에서 스팸 메일 분류하기와 IMDB 리뷰 감성 분류하기 문제가 이진 분류 문제에 해당되며, 로이터 뉴스 분류하기 문제가 다중 클래스 문제에 해당됩니다.